In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR


In [ ]:
BASE_DIR = r'D:\Projects\elec'

In [ ]:
df = pd.read_parquet(BASE_DIR+'\\all_elec_gen_month.parquet')
df_pivot = pd.read_parquet(BASE_DIR+'\\all_elec_gen_month_pivot.parquet')

In [ ]:
def plot_generation_panels(
    df_pivot,
    start="2001-01-01",
    end="2025-09-01",
    smoothing_window=24,
    renewables=None,
    fossils=None,
):

    if renewables is None:
        renewables = [
            ("SUN", "All Utility-Scale Solar"),
            ("WND", "Wind"),
        ]
    if fossils is None:
        fossils = [
            ("PEL", "Petroleum Liquids"),
            ("NG",  "Natural Gas"),
        ]

    sub = df_pivot.loc[start:end]

    num_cols = sub.select_dtypes(include="number").columns
    smooth_num = sub[num_cols].rolling(
        window=smoothing_window,
        min_periods=1,
        center=True
    ).mean()

    smooth = sub.copy()
    smooth[num_cols] = smooth_num

    fig, ax = plt.subplots(2, 2, figsize=(11, 7))

    for col, label in renewables:
        if col in smooth.columns:
            ax[0,0].plot(smooth.index, smooth[col], label=label)
    ax[0,0].set_title("Renewable Generation")
    ax[0,0].legend()

    for col, label in fossils:
        if col in smooth.columns:
            ax[0,1].plot(smooth.index, smooth[col], label=label)
    ax[0,1].set_title("Fossil Generation")
    ax[0,1].legend()

    for col, label in renewables + fossils:
        if col in smooth.columns:
            ax[1,0].plot(smooth.index, smooth[col], label=label)
    ax[1,0].set_title("Combined Generation")

    ax[1,1].axis("off")
    handles, labels = ax[1,0].get_legend_handles_labels()
    ax[1,1].legend(handles, labels, loc="center", fontsize=9)

    for r in [0, 1]:
        for c in [0, 1]:
            if r == 1 and c == 1:
                continue  

            axis = ax[r, c]
            axis.grid(True, alpha=0.3)
            axis.tick_params(axis="x", labelrotation=45)
            axis.set_xlabel(f"{start} to {end}")
            axis.set_ylabel("MWh")

            axis.xaxis.set_major_locator(mdates.YearLocator(5))     
            axis.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_displacement(
    df_pivot,
    start="2001-01-01",
    end="2025-09-01",
    smoothing_window=24,
    displace_window=36,
    renewables=None,
    fossils=None,
):


    
    if renewables is None:
        renewables = [
            ("SUN", "All Utility-Scale Solar"),
            ("WND", "Wind"),
        ]
    if fossils is None:
        fossils = [
            ("PEL", "Petroleum Liquids"),
            ("NG",  "Natural Gas"),
        ]

    sub = df_pivot.loc[start:end]

    num_cols = sub.select_dtypes(include="number").columns
    smooth_num = sub[num_cols].rolling(
        window=smoothing_window,
        min_periods=1,
        center=True
    ).mean()

    smooth = sub.copy()
    smooth[num_cols] = smooth_num

    renew = smooth['SUN'] + smooth['WND']
    gas   = smooth['NG']
    oil   = smooth['PEL']
    coal = smooth['COW']
    dRenew = renew.diff()
    dGas   = gas.diff()
    dOil   = oil.diff()
    dCoal = coal.diff()
    corr_gas = dRenew.corr(dGas)
    corr_oil = dRenew.corr(dOil)
    corr_coal = dRenew.corr(dCoal)
    rc_gas = dRenew.rolling(displace_window).corr(dGas)
    rc_oil = dRenew.rolling(displace_window).corr(dOil)
    rc_coal = dRenew.rolling(displace_window).corr(dCoal)

    
    fig, ax = plt.subplots(figsize=(10,5))

    ax.plot(rc_gas, label='Renewables vs Gas')
    ax.plot(rc_oil, label='Renewables vs Petroleum')
    ax.plot(rc_coal, label='Renewables vs Coal')
    
    ax.axhline(0, color='black', linestyle='--', alpha=0.5)
    ax.axvline(16450, color='black', linestyle='--', alpha=0.5)
    ax.set_title("Rolling Correlation — Renewable Growth vs Fossil Fuel Changes")
    ax.set_ylabel("Correlation (Negative = Displacement)")
    ax.legend()
    ax.grid(alpha=0.3)
    plt.show()


In [ ]:
def plot_granger(df, title):
    plt.figure(figsize=(10,4))
    
    # Plot p-values
    plt.plot(df['Lag'], df['F-test P-value'], marker='o', label='P-value')
    
    # Add significance threshold line
    plt.axhline(0.05, color='red', linestyle='--', linewidth=1, label='0.05 threshold')
    
    # Log scale helps show extremely small values
    plt.yscale('log')
    
    plt.xlabel("Lag")
    plt.ylabel("F-test P-value (log scale)")
    plt.title(title)
    plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_generation_panels(
    df_pivot,
    start="2001-01-01",
    end="2025-09-01",
    smoothing_window=12,
    renewables=['SUN','WND'],
    fossils=['PEL','COW','NG']
):
    sub = df_pivot.loc[start:end]

    num_cols = sub.select_dtypes(include="number").columns
    smooth_num = sub[num_cols].rolling(
        window=smoothing_window,
        min_periods=1,
        center=True
    ).mean()

    smooth = sub.copy()
    smooth[num_cols] = smooth_num

    fig, ax = plt.subplots(2, 2, figsize=(11, 7))

    for col, label in renewables:
        if col in smooth.columns:
            ax[0,0].plot(smooth.index, smooth[col], label=label)
    ax[0,0].set_title("Renewable Generation")
    ax[0,0].legend()

    for col, label in fossils:
        if col in smooth.columns:
            ax[0,1].plot(smooth.index, smooth[col], label=label)
    ax[0,1].set_title("Coal Generation")
    ax[0,1].legend()

    for col, label in renewables + fossils:
        if col in smooth.columns:
            ax[1,0].plot(smooth.index, smooth[col], label=label)
    ax[1,0].set_title("Combined Generation")

    ax[1,1].axis("off")
    handles, labels = ax[1,0].get_legend_handles_labels()
    ax[1,1].legend(handles, labels, loc="center", fontsize=9)

    for r in [0, 1]:
        for c in [0, 1]:
            if r == 1 and c == 1:
                continue  

            axis = ax[r, c]
            axis.grid(True, alpha=0.3)
            axis.tick_params(axis="x", labelrotation=45)
            axis.set_xlabel(f"{start} to {end}")
            axis.set_ylabel("MWh")

            axis.xaxis.set_major_locator(mdates.YearLocator(5))     
            axis.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    plt.tight_layout()
    plt.show()


In [ ]:
## Define data sources
solar_util=("SUN", "solar_utility")
solar_pv_util=("SPV", "solar_pv_utility")
solar_therm=("STH", "solar_thermal_utility")
solar_dist=("DPV", "solar_pv_distributed")
solar_all=("TSN", "solar_all")
wind=("WND", "wind")
hydro_conv=("HYC", "hydro_conventional")
hydro_pumped=("HPS", "hydro_pumped")
geotherm=("GEO", "geothermal")
wood=("WWW", "biomass_wood")
waste=("WAS", "waste")
other_renew=("AOR", "other_renewables")
coal=("COW", "coal")
gas=("NG", "natural_gas")
liquids=("PEL", "petroleum_liquids")
coke=("PC", "petroleum_coke")
other_gas=("OOG", "other_gas")
nuclear=("NUC", "nuclear")
all_fuel = ("ALL", "all_fuels")
other=("OTH", "other")

In [ ]:
## PLOT OF RENEWABLES AND FOSSILS AND COMBINED SMOOTHED
start = '2014-01-01'
end = '2024-12-31'
smoothing = 6

plot_generation_panels(df_pivot = df_pivot,
                       start=start,
                       end = end,
                       smoothing_window=smoothing,
                       renewables=[solar_util],
                       fossils=[gas ,coal]
                      )



In [ ]:
## ROLLING CORRELATION
start = '2002-01-01'
end = '2024-09-01'
smoothing = 12
displace_window=72

plot_displacement(df_pivot = df_pivot,
                       start=start,
                       end = end,
                       smoothing_window=smoothing,
                       displace_window= displace_window,
                       renewables=[solar_util, wind],
                       fossils=[gas, coal]
                      )


In [ ]:
## Displacement Index

start = '2001-01-01'
end = '2025-09-01'
smoothing_window = 12
displace_window=60


sub = df_pivot.loc[start:end]

num_cols = sub.select_dtypes(include="number").columns
smooth_num = sub[num_cols].rolling(
    window=smoothing_window,
    min_periods=1,
    center=True
).mean()

smooth = sub.copy()
smooth[num_cols] = smooth_num

renew = smooth['SUN'] + smooth['WND']
gas   = smooth['NG']
oil   = smooth['PEL']
coal = smooth['COW']
dRenew = renew.diff()
dGas   = gas.diff()
dOil   = oil.diff()
dCoal = coal.diff()
corr_gas = dRenew.corr(dGas)
corr_oil = dRenew.corr(dOil)
corr_coal = dRenew.corr(dCoal)
rc_gas = dRenew.rolling(displace_window).corr(dGas)
rc_oil = dRenew.rolling(displace_window).corr(dOil)
rc_coal = dRenew.rolling(displace_window).corr(dCoal)

disp_gas = -1*rc_gas
disp_oil = -1*rc_oil
disp_coal = -1*rc_coal

disp_df = pd.DataFrame({
    "Gas": disp_gas,
    "Oil": disp_oil,
    "Coal": disp_coal,
})
ax = disp_df.plot(figsize=(10,5), title="Displacement Correlated to Increasing Renewable Output")
ax.set_ylabel('(Positive is Positive Displacement)')
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.axvline('2015-01-01', color='black', linestyle='--', alpha=0.5)
ax.legend(['Natural Gas', 'Petroleum', 'Coal'])
plt.show()

In [ ]:
## Conditional Regression
renew = df_pivot['SUN'] + df_pivot['WND']   
gas   = df_pivot['NG']
coal  = df_pivot['COW']
total_gen = df_pivot[['SUN','WND','NG','COW']].sum(axis=1)
ts = pd.DataFrame({
    'renew': renew,
    'gas': gas,
    'coal': coal,
    'total': total_gen
}).dropna()

# Month by month changes
dts = ts.diff().dropna()

##### CHANGE FOR GAS AND COAL COMPARISONS
y = dts['gas']
X = dts[['renew', 'total']]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())



In [ ]:
## Granger Causality
renew = df_pivot['SUN'] + df_pivot['WND']
gas   = df_pivot['NG']
coal  = df_pivot['COW']

ts = pd.DataFrame({
    'renew': renew,
    'gas': gas,
    'coal': coal,
}).dropna()

# Month by month changes
dts = ts.diff().dropna()
# Smooth out extreme noise
dts = dts.rolling(3, center=True).mean().dropna()
ftest_gas=[]
data = dts[['gas', 'renew']]
print('Results for renewables --> gas')
granger_results_gas = grangercausalitytests(data, maxlag=12, verbose=True)
print('='*100)
for lag in granger_results_gas:
    test_statistic, p_value,new, df = granger_results_gas[lag][0]['ssr_ftest']
    ftest_gas.append([lag,p_value])
ftest_coal=[]
data = dts[['coal', 'renew']]
print('Results for renewables --> coal')
granger_results_coal = grangercausalitytests(data, maxlag=12, verbose=True)
print('='*100)
for lag in granger_results_coal:
    test_statistic, p_value,new, df = granger_results_coal[lag][0]['ssr_ftest']
    ftest_coal.append([lag,p_value])

In [ ]:
## Plot Grangers

ftest_gas = pd.DataFrame(ftest_gas,columns=['Lag','F-test P-value'])
ftest_coal = pd.DataFrame(ftest_coal,columns=['Lag','F-test P-value'])
plot_granger(ftest_gas,        "Granger Test: Renewables → Gas")
plot_granger(ftest_coal,       "Granger Test: Renewables → Coal")

In [ ]:
## VAR Impulse-Response
renew = df_pivot['SUN'] + df_pivot['WND']
gas   = df_pivot['NG']
coal  = df_pivot['COW']

ts = pd.DataFrame({
    'renew': renew,
    'gas': gas,
    'coal': coal,
}).dropna()

dts = ts.diff().dropna() 
#dts['renew'] = dts['SUN'] + dts['WND']
var_df = dts[['renew', 'gas', 'coal']].dropna()

model = VAR(var_df)
results = model.select_order(12)
print(results.summary())
var_res = model.fit(maxlags=4)
print(var_res.summary())
irf = var_res.irf(24)   
irf.plot(orth=True)
cum = irf.cum_effects
renew_idx = list(var_df.columns).index('renew')
gas_idx   = list(var_df.columns).index('gas')

cum_renew_gas = cum[:, gas_idx, renew_idx]
plt.figure(figsize=(10,5))
plt.plot(cum_renew_gas, label="Cumulative IRF: renew → gas", color='blue')
plt.axhline(0, color='black', linestyle='--', linewidth=1)

plt.xlabel("Months after shock")
plt.ylabel("Cumulative change in gas (MWh)")
plt.title("Cumulative Impulse Response: Renewables → Natural Gas")
plt.grid(alpha=0.3)
plt.legend()
plt.show()